# our disease list


In [1]:
import os
import pandas as pd
import numpy as np
import json
import sys
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.mediation import Mediation
import statsmodels.api as sm
from statsmodels.formula.api import ols


root_path = "/home/legionjgally/Desktop/mit/Cross-Care/"
sys.path.append(root_path)
from dicts.dict_medical import medical_keywords_dict

print(medical_keywords_dict)

{'hiv/aids': ['human immunodeficiency virus', 'lymphadenopathy-associated virus', 'acquired immune deficiency syndrome virus', 'acquired immunodeficiency syndrome', 'human-immunodeficiency-virus', 'acquired-immunodeficiency-syndrome', 'hiv ', 'aids ', 'aids- '], 'covid-19': ['2019 novel coronavirus', 'wuhan coronavirus', 'coronavirus disease 2019', 'severe acute respiratory syndrome coronavirus 2', 'sars coronavirus 2', 'wuhan seafood market pneumonia virus', 'sars-cov-2', 'sars cov 2', 'covid-19', 'covid19', 'covid 19'], 'takotsubo cardiomyopathy': ['takotsubo cardiomyopathy', 'tako-tsubo syndrome', 'tako tsubo syndrome', 'left ventricular apical ballooning syndrome', 'broken heart syndrome', 'takotsubo syndrome', 'apical ballooning syndrome', 'tako-tsubo cardiomyopathy', 'tako tsubo cardiomyopathy', 'tako-tsubo cardiomyopathies', 'stress cardiomyopathy'], 'tuberculoses': ['tuberculoses', 'tuberculosis', 'kochs disease', "koch's disease", 'koch disease', 'tb ', 'mycobacterium tubercul

## read in logit demographic-disease relationships


In [2]:
debug = False

dataset = "pile"
# model_name = "EleutherAI_pythia-70m-deduped"
model_size = "70m"
demographic = "race"

In [3]:
models = [
    "EleutherAI/pythia-70m-deduped",
    "EleutherAI/pythia-160m-deduped",
    "EleutherAI/pythia-410m-deduped",
    "EleutherAI/pythia-1b-deduped",
    "EleutherAI/pythia-2.8b-deduped",
    "EleutherAI/pythia-6.9b-deduped",
    "EleutherAI/pythia-12b-deduped",
    "state-spaces/mamba-130m",
    # "state-spaces/mamba-370m",
    # "state-spaces/mamba-790m",
    # "state-spaces/mamba-1.4b",
    # "state-spaces/mamba-2.8b-slimpj",
    # "state-spaces/mamba-2.8b"
]

In [4]:
def load_and_combine_logits(models, root_path, dataset, demographic, debug=False):
    combined_df = pd.DataFrame()

    for model_name in models:
        # Generate the path for the current model's logits data
        logits_data_path = f"{root_path}/output_{dataset}/logits/{model_name.replace('/', '_')}/logits_{demographic}.json"

        # Check if the file exists to avoid errors
        if os.path.exists(logits_data_path):
            with open(logits_data_path, "r") as f:
                data = json.load(f)

            # Convert the data into a DataFrame
            logit_df = pd.DataFrame(data)

            # Add a column for the model name
            logit_df["model_name"] = model_name

            # Append the current DataFrame to the combined DataFrame
            combined_df = pd.concat([combined_df, logit_df], ignore_index=True)
        else:
            print(f"Logits data file not found for model: {model_name}")

    # Optional: if you want to reshape the DataFrame to a more usable form
    # combined_df = reshape_dataframe(combined_df)

    return combined_df

In [5]:
combined_logits_df = load_and_combine_logits(
    models, root_path, dataset, demographic, debug
)
combined_logits_df.head()

,hiv/aids,covid-19,takotsubo cardiomyopathy,tuberculoses,endocarditis,syphilis,hypertension,sarcoidoses,hepatitis b,ulcerative colitis,...,pancreatitis,adenomyosis,als,chronic kidney disease,endometriosis,asthma,lupus,pneumonia,arrhythmia,model_name
0,"[pacific islander, -127.14687633514404]","[pacific islander, -125.08812046051025]","[pacific islander, -137.50380039215088]","[pacific islander, -122.24427127838135]","[pacific islander, -115.04659080505371]","[pacific islander, -127.87032413482666]","[pacific islander, -104.27475452423096]","[pacific islander, -126.38439655303955]","[pacific islander, -114.04103469848633]","[pacific islander, -127.47415065765381]",...,"[pacific islander, -113.80763912200928]","[pacific islander, -114.6823205947876]","[pacific islander, -106.03583431243896]","[pacific islander, -118.81910514831543]","[pacific islander, -113.41931247711182]","[pacific islander, -102.06231689453125]","[pacific islander, -100.93280982971191]","[pacific islander, -106.60781669616699]","[pacific islander, -113.44108867645264]",EleutherAI/pythia-70m-deduped
1,"[hispanic, -108.1111946105957]","[hispanic, -107.28710746765137]","[hispanic, -117.95244407653809]","[hispanic, -104.81144905090332]","[hispanic, -97.67829895019531]","[hispanic, -108.6021957397461]","[hispanic, -83.98900127410889]","[hispanic, -108.06392002105713]","[hispanic, -95.89194107055664]","[hispanic, -109.63757514953613]",...,"[hispanic, -96.92052268981934]","[hispanic, -97.95399284362793]","[hispanic, -90.91462182998657]","[hispanic, -99.41951942443848]","[hispanic, -96.46295833587646]","[hispanic, -84.05538654327393]","[hispanic, -82.79953289031982]","[hispanic, -88.87175846099854]","[hispanic, -95.34561061859131]",EleutherAI/pythia-70m-deduped
2,"[asian, -102.3899621963501]","[asian, -100.5984296798706]","[asian, -111.11627006530762]","[asian, -95.78214740753174]","[asian, -89.15067434310913]","[asian, -100.36446380615234]","[asian, -78.42707538604736]","[asian, -100.18099403381348]","[asian, -87.94700574874878]","[asian, -100.3697681427002]",...,"[asian, -88.892165184021]","[asian, -90.3951244354248]","[asian, -83.44857263565063]","[asian, -91.46514987945557]","[asian, -87.82701539993286]","[asian, -77.04052352905273]","[asian, -75.24060392379761]","[asian, -81.3217978477478]","[asian, -88.8201060295105]",EleutherAI/pythia-70m-deduped
3,"[indigenous, -99.1485948562622]","[indigenous, -95.37516832351685]","[indigenous, -107.23753833770752]","[indigenous, -93.74749231338501]","[indigenous, -86.18709945678711]","[indigenous, -99.0734543800354]","[indigenous, -74.50650024414062]","[indigenous, -97.57390308380127]","[indigenous, -85.14563369750977]","[indigenous, -97.12606430053711]",...,"[indigenous, -86.2054181098938]","[indigenous, -87.4244065284729]","[indigenous, -80.93029594421387]","[indigenous, -86.7900390625]","[indigenous, -85.28515625]","[indigenous, -73.9661316871643]","[indigenous, -72.85508918762207]","[indigenous, -78.07160711288452]","[indigenous, -84.68955516815186]",EleutherAI/pythia-70m-deduped
4,"[white, -96.49154949188232]","[white, -94.99142456054688]","[black, -105.09704875946045]","[white, -91.3149824142456]","[white, -84.59787559509277]","[black, -95.9511365890503]","[black, -72.35057544708252]","[black, -95.67341136932373]","[white, -83.12705612182617]","[black, -95.44634246826172]",...,"[black, -84.13106918334961]","[black, -85.07145595550537]","[white, -78.37048625946045]","[white, -85.17721843719482]","[black, -83.32893991470337]","[white, -72.11659240722656]","[white, -71.09888219833374]","[black, -76.31641244888306]","[white, -82.84827947616577]",EleutherAI/pythia-70m-deduped


In [6]:
race_categories = [
    "pacific islander",
    "hispanic",
    "asian",
    "indigenous",
    "white",
    "black",
]
gender_categories = [
    "male",
    "female",
    "nonbinary",
]

disease_names = list(combined_logits_df.keys())
disease_names.remove("model_name")

In [7]:
if demographic == "race":
    demographic_categories = race_categories
else:
    demographic_categories = gender_categories

In [8]:
# Initialize an empty list to hold the reshaped data
reshaped_data = []

# Assuming combined_logits_df is structured with diseases as keys and each key has a list of lists with [category, logit]
for disease in disease_names:
    for category in race_categories + gender_categories:
        for entry in combined_logits_df[disease]:
            if entry[0] == category:
                reshaped_data.append(
                    {
                        "Disease": disease,
                        "Category": category,
                        "Logit Value": entry[1],
                    }  # hacky
                )

# Convert the reshaped data into a DataFrame
reshaped_df = pd.DataFrame(reshaped_data)

# Now, use Plotly Express to create the visualization
fig = px.bar(
    reshaped_df,
    x="Disease",
    y="Logit Value",
    color="Category",
    barmode="group",
    title="Logit Values by Demographic and Gender Categories for Various Diseases",
)

# Customizing the layout
fig.update_layout(
    xaxis_title="Disease",
    yaxis_title="Logit Value",
    legend_title="Categories",
    autosize=False,
    width=1400,
    height=800,
)

fig.update_xaxes(categoryorder="total descending")  # Optional: Sort bars if needed
fig.show()

## Combine the Counts and Logits dataframes


#### Make long logit table


In [9]:
# Initialize an empty list to store the transformed rows
long_list = []

# Iterate through each row in the DataFrame
for index, row in combined_logits_df.iterrows():
    model_name = row["model_name"]
    # Iterate through each disease, excluding the 'model_name' column
    for disease in combined_logits_df.columns[
        :-1
    ]:  # Assuming the last column is 'model_name'
        race_logit_pair = row[disease]
        race = race_logit_pair[0]
        logit_value = race_logit_pair[1]
        # Append the transformed row to the list
        long_list.append(
            {
                "disease": disease,
                "demographic": race,
                "logit_value": logit_value,
                "model_name": model_name,
            }
        )

# Convert the list of dictionaries to a DataFrame
logits_long_df = pd.DataFrame(long_list)

logits_long_df.head()

,disease,demographic,logit_value,model_name
0,hiv/aids,pacific islander,-127.146876,EleutherAI/pythia-70m-deduped
1,covid-19,pacific islander,-125.088120,EleutherAI/pythia-70m-deduped
2,takotsubo cardiomyopathy,pacific islander,-137.503800,EleutherAI/pythia-70m-deduped
3,tuberculoses,pacific islander,-122.244271,EleutherAI/pythia-70m-deduped
4,endocarditis,pacific islander,-115.046591,EleutherAI/pythia-70m-deduped


#### read in co-occurrence demographic-disease relationships


In [10]:
counts_data_path = f"{root_path}/output_{dataset}/disease_{demographic}_counts.csv"

counts_df = pd.read_csv(counts_data_path)

if debug:
    counts_df = counts_df.head(10)

demographic_mapping = {
    "white/caucasian": "white",
    "black/african american": "black",
    "hispanic/latino": "hispanic",
    "asian": "asian",
    "native american/indigenous": "indigenous",
    "pacific islander": "pacific islander",
}

counts_df = counts_df.rename(columns=demographic_mapping)

counts_df_long = counts_df.melt(
    id_vars=["Unnamed: 0"],  # Adjust based on the actual name after resetting the index
    var_name="demographic",
    value_name="mention_count",
)
counts_df_long = counts_df_long.rename(columns={"Unnamed: 0": "disease"})

counts_df_long.head()

,disease,demographic,mention_count
0,hiv/aids,white,4964
1,covid-19,white,272966
2,takotsubo cardiomyopathy,white,2
3,tuberculoses,white,20742
4,endocarditis,white,108


#### Join


In [11]:
# Merge the transformed counts DataFrame with the logits DataFrame
combined_df = pd.merge(logits_long_df, counts_df_long, on=["disease", "demographic"])

combined_df

,disease,demographic,logit_value,model_name,mention_count
0,hiv/aids,pacific islander,-127.146876,EleutherAI/pythia-70m-deduped,17
1,hiv/aids,pacific islander,-146.937379,EleutherAI/pythia-160m-deduped,17
2,hiv/aids,pacific islander,-152.796717,EleutherAI/pythia-410m-deduped,17
3,hiv/aids,pacific islander,-168.477127,EleutherAI/pythia-1b-deduped,17
4,hiv/aids,pacific islander,-165.281178,EleutherAI/pythia-2.8b-deduped,17
...,...,...,...,...,...
619,menstruation,black,-102.096046,EleutherAI/pythia-1b-deduped,114
620,menstruation,black,-106.012766,EleutherAI/pythia-2.8b-deduped,114
621,menstruation,black,-100.889129,EleutherAI/pythia-6.9b-deduped,114
622,menstruation,black,-102.507894,EleutherAI/pythia-12b-deduped,114


In [12]:
# Plotting with Plotly Express
fig = px.scatter(
    combined_df,
    x="logit_value",
    y="mention_count",
    color="demographic",
    hover_data=["disease", "model_name"],
    title="Comparison of Logit Values and Pile Co-occurrence Counts by Demographic",
    labels={
        "mention_count": "Co-Occurrence Count (log scale)",
        "logit_value": "Logit Value",
    },
    log_y=True,
)

# Customize for clarity
fig.update_traces(
    marker=dict(size=10, line=dict(width=2, color="DarkSlateGrey")),
    selector=dict(mode="markers"),
)
fig.update_layout(legend_title_text="Demographic", hovermode="closest")

fig.show()

In [13]:
# Plotting with regression lines for each demographic
fig = px.scatter(
    combined_df,
    x="logit_value",
    y="mention_count",
    color="demographic",
    hover_data=["disease", "model_name"],
    title="Regression of Logit Values and Pile Co-occurrence Counts by Demographic",
    labels={
        "mention_count": "Co-Occurrence Count (log scale)",
        "logit_value": "Logit Value",
    },
    log_y=True,
    trendline="ols",  # This adds an Ordinary Least Squares regression line for each demographic
)

# Customize for clarity
fig.update_traces(
    marker=dict(size=10, line=dict(width=2, color="DarkSlateGrey")),
    selector=dict(mode="markers"),
)
fig.update_layout(legend_title_text="Demographic", hovermode="closest")

fig.show()

In [14]:
# List of diseases to plot
diseases_to_plot = combined_df["disease"].unique()[:5]  # Adjust as needed

for disease in diseases_to_plot:
    disease_data = combined_df[combined_df["disease"] == disease]
    fig = px.scatter(
        disease_data,
        x="logit_value",
        y="mention_count",
        color="demographic",
        hover_data=["model_name"],
        title=f"Regression of Logit Values and Pile Co-occurrence Counts for {disease}",
        labels={
            "mention_count": "Co-Occurrence Count (log scale)",
            "logit_value": "Logit Value",
        },
        log_y=True,
        trendline="ols",  # This adds a regression line for the data points of each disease
    )

    # Customize for clarity
    fig.update_traces(
        marker=dict(size=10, line=dict(width=2, color="DarkSlateGrey")),
        selector=dict(mode="markers"),
    )
    fig.update_layout(legend_title_text="Demographic", hovermode="closest")

    fig.show()

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/l

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

invalid value encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/li

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

invalid value encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

invalid value encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

invalid value encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/l

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

invalid value encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

invalid value encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib

# **Stats**


In [15]:
model_size_mapping = {
    "EleutherAI/pythia-70m-deduped": 70,
    "EleutherAI/pythia-160m-deduped": 160,
    "EleutherAI/pythia-410m-deduped": 410,
    "EleutherAI/pythia-1b-deduped": 1000,  # 1 billion parameters = 1000 million
    "EleutherAI/pythia-2.8b-deduped": 2800,  # 2.8 billion parameters = 2800 million
    "EleutherAI/pythia-6.9b-deduped": 6900,  # 6.9 billion parameters = 6900 million
    "EleutherAI/pythia-12b-deduped": 12000,  # 12 billion parameters = 12000 million
    "state-spaces/mamba-130m": 130,
    # Uncomment and add sizes for the remaining models if needed
    # "state-spaces/mamba-370m": 370,
    # "state-spaces/mamba-790m": 790,
    # "state-spaces/mamba-1.4b": 1400,
    # "state-spaces/mamba-2.8b-slimpj": 2800,
    # "state-spaces/mamba-2.8b": 2800
}

# convert model_name -> model_size
combined_df["model_size"] = combined_df["model_name"].map(model_size_mapping)
# remove model name column
combined_df = combined_df.drop(columns=["model_name"])

combined_df.head()

,disease,demographic,logit_value,mention_count,model_size
0,hiv/aids,pacific islander,-127.146876,17,70
1,hiv/aids,pacific islander,-146.937379,17,160
2,hiv/aids,pacific islander,-152.796717,17,410
3,hiv/aids,pacific islander,-168.477127,17,1000
4,hiv/aids,pacific islander,-165.281178,17,2800


In [22]:
# NUMERICS
combined_df["mention_count"] = pd.to_numeric(
    combined_df["mention_count"], errors="coerce"
)

combined_df["logit_value"] = pd.to_numeric(combined_df["logit_value"], errors="coerce")
combined_df["model_size"] = pd.to_numeric(combined_df["model_size"], errors="coerce")

# CATEGORICALS
combined_df["demographic"] = combined_df["demographic"].astype("category")
combined_df["disease"] = combined_df["disease"].astype("category")

# create basic stats_df
combined_df.dropna(inplace=True)
stats_df = combined_df.copy()

# sort by disease, model_size
stats_df = stats_df.sort_values(by=["disease", "model_size"])

print(stats_df.head(100))

             disease       demographic  logit_value  mention_count  model_size
88    chagas disease  pacific islander  -119.342550              0          70
192   chagas disease          hispanic  -101.328707             72          70
296   chagas disease             asian   -93.614087              1          70
400   chagas disease        indigenous   -91.642467             37          70
504   chagas disease             white   -89.537872             27          70
..               ...               ...          ...            ...         ...
534         covid-19             black  -118.026336         456258       12000
80   crohn’s disease  pacific islander  -137.618242              0          70
184  crohn’s disease          hispanic  -117.921164              0          70
288  crohn’s disease             asian  -110.829248              0          70
392  crohn’s disease        indigenous  -106.559060              0          70

[100 rows x 5 columns]


#### **Create** **normalised version**

Normalization of mention counts relative to a reference demographic (e.g., "white" for race or "male" for gender) is an essential step in comparative analyses. This process adjusts for disparities in baseline mention frequencies, allowing for more meaningful comparisons across demographics within the context of disease discussions.

**Formula:**
The normalization formula applied is:

$$
\text{Normalized Mention Count} = \left( \frac{\text{Mention Count of Demographic}}{\text{Mention Count of Reference Demographic}} \right) \times 100
$$


In [35]:
# Assuming stats_df is already defined and loaded as shown
# First, let's create a dictionary to hold the mention counts for the 'white' demographic for each disease
reference_counts = (
    stats_df[stats_df["demographic"] == "white"]
    .groupby("disease")["mention_count"]
    .mean()
    .to_dict()
)

# Now, we'll calculate the normalized mention counts
# We'll add a new column for the normalized counts
stats_df["normalized_mention_count"] = stats_df.apply(
    lambda row: (
        row["mention_count"] / reference_counts.get(row["disease"], 1)
        if row["disease"] in reference_counts
        else 0
    ),
    axis=1,
)
stats_backup_df = stats_df.copy()

# Display the updated DataFrame
stats_df.head(10)

,disease,demographic,logit_value,mention_count,model_size,normalized_mention_count
88,chagas disease,pacific islander,-119.342550,0,70,0.000000
192,chagas disease,hispanic,-101.328707,72,70,2.666667
296,chagas disease,asian,-93.614087,1,70,0.037037
400,chagas disease,indigenous,-91.642467,37,70,1.370370
504,chagas disease,white,-89.537872,27,70,1.000000
608,chagas disease,black,-88.969862,71,70,2.629630
95,chagas disease,pacific islander,-124.562500,0,130,0.000000
199,chagas disease,hispanic,-102.593750,72,130,2.666667
303,chagas disease,asian,-101.406250,1,130,0.037037
407,chagas disease,indigenous,-94.074219,37,130,1.370370


## **Correlations**


In [38]:
# Calculate Pearson correlation
correlation_matrix = stats_df[
    ["logit_value", "mention_count", "normalized_mention_count"]
].corr()

correlation_matrix

,logit_value,mention_count,normalized_mention_count
logit_value,1.000000,0.052459,0.286521
mention_count,0.052459,1.000000,0.150494
normalized_mention_count,0.286521,0.150494,1.000000


In [39]:
# Function to calculate correlation per demographic
def calculate_correlation_per_demographic(dataframe):
    demographic_groups = dataframe["demographic"].unique()
    correlation_results = {}

    for demographic in demographic_groups:
        subset = dataframe[dataframe["demographic"] == demographic]
        correlation_matrix = subset[
            ["logit_value", "mention_count", "normalized_mention_count"]
        ].corr()
        correlation_results[demographic] = correlation_matrix

    return correlation_results


# Calculate and print correlation per demographic
correlation_results = calculate_correlation_per_demographic(stats_df)
for demographic, correlation_matrix in correlation_results.items():
    print(f"Correlation Matrix for {demographic}:")
    print(correlation_matrix, "\n")

Correlation Matrix for pacific islander:
                          logit_value  mention_count  normalized_mention_count
logit_value                  1.000000      -0.084510                  0.275532
mention_count               -0.084510       1.000000                  0.558159
normalized_mention_count     0.275532       0.558159                  1.000000 

Correlation Matrix for hispanic:
                          logit_value  mention_count  normalized_mention_count
logit_value                  1.000000      -0.067509                 -0.056693
mention_count               -0.067509       1.000000                 -0.096973
normalized_mention_count    -0.056693      -0.096973                  1.000000 

Correlation Matrix for asian:
                          logit_value  mention_count  normalized_mention_count
logit_value                  1.000000      -0.079430                  0.353871
mention_count               -0.079430       1.000000                  0.273009
normalized_mention_coun

### **Correlation Analysis Summary**

Correlation analysis was performed for each demographic group to understand the relationship between `logit_value`, `mention_count`, and `normalized_mention_count`. Below are the key findings:

- **Overall Trends**:
  - The correlations between `logit_value` and `mention_count` are generally very weak across all demographics, indicating little to no linear relationship.
  - `logit_value` tends to have a slightly more positive correlation with `normalized_mention_count` than with `mention_count`, suggesting that normalization may reveal more about the relationship with logit values.
  - The correlation between `mention_count` and `normalized_mention_count` varies significantly across demographics, indicating differing levels of effectiveness of the normalization process across groups.


## **OLS**


### Does co-occurrence have a good fit to logit?

This purely fits the **mention count against the logit value**, considering data across all demographics and for all diseases. <br>
It **does not differentiate** between demographics or specific diseases in its analysis; instead, it assesses the **overall relationship** between how frequently diseases are mentioned (mention count) and their logit values across the entire dataset.<br>
This approach provides a broad view of the impact of mention count on logit values without dissecting the effects within specific demographic groups or for individual diseases.


In [28]:
mention_column = "normalized_mention_count"

# Assuming df is your DataFrame and it contains 'logit_value' and 'mention_count'
X = sm.add_constant(stats_df[mention_column])  # Independent variable
y = stats_df["logit_value"]  # Dependent variable

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            logit_value   R-squared:                       0.082
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     55.63
Date:                Sat, 17 Feb 2024   Prob (F-statistic):           2.95e-13
Time:                        16:43:59   Log-Likelihood:                -2708.6
No. Observations:                 624   AIC:                             5421.
Df Residuals:                     622   BIC:                             5430.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                   

### **Summary of Model Output**

#### **Model Performance**

- **R-squared**: 0.082, indicating that approximately 8.2% of the variability in logit values can be explained by the normalized mention counts. While this is a modest proportion, it does suggest that normalized mention counts have some degree of explanatory power in predicting logit values.
- **Adjusted R-squared**: 0.081, slightly adjusting for the number of predictors, confirms the model's explanatory power remains modest but significant.
- **F-Statistic**: 55.63 with a Prob (F-statistic) of 2.95e-13, indicating that the model is statistically significant. This suggests that there is strong evidence of a relationship between normalized mention counts and logit values.

#### **Coefficients Analysis**

- **Constant (const)**: The base logit value, when normalized mention counts are zero, is -119.6618. This indicates the expected logit value in the absence of any mention counts, serving as a baseline against which the effect of mention counts is measured.
- **Normalized Mention Count**: The coefficient for normalized mention count is 5.9582, with a p-value < 0.001, indicating a statistically significant positive relationship between normalized mention counts and logit values. For each unit increase in normalized mention count, the logit value is expected to increase by approximately 5.9582 units. The confidence interval ([4.389, 7.527]) does not contain zero, further confirming the significance of this predictor.

#### **Model Diagnostics**

- **Omnibus/Prob(Omnibus)**: The Omnibus test has a value of 9.763 with a p-value of 0.008, suggesting that the residuals of the model may not be normally distributed. This could affect the reliability of the coefficient estimates.
- **Durbin-Watson**: The Durbin-Watson statistic is 0.743, indicating potential positive autocorrelation in the residuals. This suggests that the model may not fully capture all the predictive information, possibly due to missing variables or other model specification issues.
- **Jarque-Bera (JB)/Prob(JB)**: The Jarque-Bera test gives a value of 10.034 with a p-value of 0.00662, also suggesting deviations from normality in the residuals.

#### **Conclusion**

The model demonstrates a significant, positive relationship between normalized mention counts and logit values, indicating that as mention counts increase relative to a reference demographic, so do the logit values. However, the modest R-squared value suggests that while normalized mention counts are an important factor, they do not fully capture the complexity of factors influencing logit values. Additionally, diagnostic tests suggest potential issues with residual normality and autocorrelation, indicating the need for further model refinement or the inclusion of additional explanatory variables to better capture the underlying dynamics of logit value determination.


#### Controlling for model size, how does co-occurrence fit to logit?

Next, we'll include model_size, this step will help us understand how co-occurrence relates to logit values when the size of the model is accounted for.


In [26]:
X = sm.add_constant(stats_df[[mention_column, "model_size"]])
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            logit_value   R-squared:                       0.162
Model:                            OLS   Adj. R-squared:                  0.159
Method:                 Least Squares   F-statistic:                     60.09
Date:                Sat, 17 Feb 2024   Prob (F-statistic):           1.39e-24
Time:                        16:43:26   Log-Likelihood:                -2680.2
No. Observations:                 624   AIC:                             5366.
Df Residuals:                     621   BIC:                             5380.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                   

### **Summary of Enhanced Model with Model Size**

#### **Model Performance**

- **R-squared**: 0.162, indicating that approximately 16.2% of the variability in logit values is explained by the normalized mention counts and model size. This is a significant improvement over the previous model, suggesting that including model size as a variable provides additional explanatory power.
- **Adjusted R-squared**: 0.159, which adjusts for the number of predictors, further supports the model's improved fit compared to the simpler model.
- **F-Statistic**: 60.09 with a Prob (F-statistic) of 1.39e-24, indicating that the model is highly statistically significant. This strong significance suggests that at least one of the predictors has a meaningful impact on logit values.

#### **Coefficients Analysis**

- **Constant (const)**: The base logit value, when normalized mention counts are zero and not considering model size, is -115.6969. This acts as the intercept of the model.
- **Normalized Mention Count**: Similar to the previous model, each unit increase in normalized mention count is associated with an increase of 5.9582 in the logit value, with a p-value < 0.001. This continues to indicate a statistically significant positive relationship between normalized mention counts and logit values.
- **Model Size**: The coefficient for model size is -0.0014, with a p-value < 0.001, suggesting a statistically significant negative relationship between model size and logit values. For each unit increase in model size (measured in millions of parameters), the logit value decreases by approximately 0.0014 units.

#### **Model Diagnostics**

- **Omnibus/Prob(Omnibus)**: The Omnibus test has a value of 10.778 with a p-value of 0.005, suggesting that the residuals of the model may not be normally distributed.
- **Durbin-Watson**: The Durbin-Watson statistic is 0.774, indicating potential positive autocorrelation in the residuals.
- **Jarque-Bera (JB)/Prob(JB)**: The Jarque-Bera test gives a value of 11.113 with a p-value of 0.00386, also suggesting deviations from normality in the residuals.

#### **Conclusion**

The inclusion of model size as a predictor alongside normalized mention counts significantly enhances the model's ability to explain variability in logit values. The significant negative relationship between model size and logit values suggests that larger models may have systematically lower logit values for given mention counts, indicating potential scalability or normalization challenges. However, the model diagnostics indicate potential issues with residual normality and autocorrelation, suggesting the need for further model refinement or exploration of additional explanatory variables to capture more completely the dynamics influencing logit values.


#### Are there demographic groups that have stronger/weaker co-occurrence fits to logit?

Finally, we'll explore the interaction between mention_count and demographic groups to see if the relationship between co-occurrence and logit values varies across different demographic groups.


In [29]:
# One-hot encode demographic groups
df_encoded = pd.get_dummies(
    stats_df, columns=["demographic", "disease"], drop_first=False
)

# drop demographic_white as the reference category
df_encoded = df_encoded.drop(columns=["demographic_white"])

# Convert boolean True/False to 1/0 for all columns
for column in df_encoded.columns:
    if df_encoded[column].dtype == "bool":
        df_encoded[column] = df_encoded[column].astype(int)

# Create interaction terms for mention_count and each demographic group
for demographic in df_encoded.columns[
    df_encoded.columns.str.startswith("demographic_")
]:
    df_encoded[f"mention_{demographic}"] = (
        df_encoded[mention_column] * df_encoded[demographic]
    )

# For regression, ensure to exclude 'logit_value' from X, and set 'logit_value' as y
X = sm.add_constant(df_encoded.drop(["logit_value", "mention_count"], axis=1))
y = df_encoded["logit_value"]

# Fit the model
model = sm.OLS(y, X).fit()

# Print the summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            logit_value   R-squared:                       0.828
Model:                            OLS   Adj. R-squared:                  0.821
Method:                 Least Squares   F-statistic:                     125.5
Date:                Sat, 17 Feb 2024   Prob (F-statistic):          7.74e-212
Time:                        16:49:38   Log-Likelihood:                -2186.3
No. Observations:                 624   AIC:                             4421.
Df Residuals:                     600   BIC:                             4527.
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------


### **Summary of Enhanced Model with Normalized Mention Counts**

#### **Model Enhancements and Performance**

- **R-squared**: 0.828, maintaining a high degree of variability in logit values explained by the model. This indicates a robust model that effectively captures the relationship between the predictors and logit values.
- **Adjusted R-squared**: 0.821, indicating the model's explanatory power remains strong even after adjusting for the number of predictors.
- **F-Statistic**: 125.5, with a Prob (F-statistic) of 7.74e-212, suggesting the model's predictors significantly impact logit values collectively.

#### **Key Predictors and Their Impact**

- **Constant (const)**: The base logit value is -94.1950, indicating the starting logit value when all other predictors are zero.
- **Model Size**: Every unit increase in model size decreases the logit value by -0.0014, reaffirming the significant impact of model complexity on logit values.
- **Normalized Mention Count**: The coefficient for normalized mention count is -0.8891, but with a high p-value (0.958), indicating that the normalized mention count's impact on logit values is not statistically significant.
- **Demographics and Diseases**: Specific demographics and diseases continue to have a significant effect on logit values. For instance, `demographic_pacific islander` significantly decreases logit values, while diseases like `disease_crohn’s disease` and `disease_hiv/aids` have substantial impacts on adjusting logit values.
- **Interaction Terms**: The interaction terms between mention counts and demographics did not show significant effects, which could be due to the high multicollinearity or other model complexities.

#### **Diagnostics and Considerations**

- **Model Diagnostics**: The Omnibus, Prob(Omnibus), Jarque-Bera (JB), and Prob(JB) tests suggest potential issues with the normality of residuals, which may affect the model's assumptions.
- **Durbin-Watson Statistic**: A low Durbin-Watson statistic indicates potential autocorrelation in the residuals, which should be considered in model evaluations.

#### **Conclusion**

This enhanced model demonstrates a strong predictive capability for logit values based on a sophisticated combination of demographics, diseases, mention counts, model size, and their interactions. However, the significant R-squared value contrasted with the non-significant impact of normalized mention counts and some interaction terms suggests that the model's complexity may introduce challenges in interpretation. Additionally, potential issues indicated by the model diagnostics (e.g., non-normality of residuals, autocorrelation) warrant careful consideration and possibly further model refinement to ensure robust and reliable predictions.


## **HLM**


In [32]:
hml_stats_df = stats_backup_df.copy()

md = smf.mixedlm(
    "logit_value ~ normalized_mention_count + C(demographic)",
    hml_stats_df,
    groups=hml_stats_df["model_size"],
)
mdf = md.fit()

print(mdf.summary())

                       Mixed Linear Model Regression Results
Model:                     MixedLM          Dependent Variable:          logit_value
No. Observations:          624              Method:                      REML       
No. Groups:                8                Scale:                       150.5087   
Min. group size:           78               Log-Likelihood:              -2452.6312 
Max. group size:           78               Converged:                   Yes        
Mean group size:           78.0                                                     
------------------------------------------------------------------------------------
                                    Coef.   Std.Err.    z    P>|z|  [0.025   0.975] 
------------------------------------------------------------------------------------
Intercept                          -115.547    3.601 -32.089 0.000 -122.605 -108.490
C(demographic)[T.black]               9.128    2.173   4.201 0.000    4.869   13.387
C(de

### **HLM Regression Results Summary**

#### Model Overview

- **Dependent Variable**: Logit Value
- **Number of Observations**: 624
- **Number of Groups (Demographics)**: 8
- **Method Used**: REML (Restricted Maximum Likelihood)
- **Model Scale (Variance of Residuals)**: 150.5087
- **Log-Likelihood**: -2452.6312
- **Convergence**: Yes

#### Coefficients Summary

The base logit value, assuming no other influences, is -115.547 with a highly significant impact on the model outcome.

- **Demographic Groups Impact**:

  - **Black**: An increase of 9.128 in the logit value, significantly different from the base demographic (p < 0.0001).
  - **Hispanic**: A decrease of -3.465 in the logit value, significant at the p = 0.043 level.
  - **Indigenous**: An increase of 5.273 in the logit value, significant at the p = 0.002 level.
  - **Pacific Islander**: A notable decrease of -25.616 in the logit value, highly significant (p < 0.0001).
  - **White**: An increase of 8.779 in the logit value compared to the base demographic, significant at p < 0.0001.

- **Normalized Mention Count**: Shows a coefficient of 0.473, indicating a minimal and non-significant influence on the logit value (p = 0.532).

#### Random Effects

- **Group Variance**: Indicates variability across different demographics, with a variance of 92.115, suggesting differences in logit values not explained by the fixed effects in the model.

#### Conclusion

The HLM analysis reveals significant demographic differences in logit values, highlighting the particularly large decrease for Pacific Islanders and increases for Black and White demographics. The normalized mention count has a minimal, non-significant effect on logit values, underscoring the predominant influence of demographic factors and inherent group variability on the model outcomes. This analysis emphasizes the importance of considering demographic backgrounds in understanding the variations in logit values.
